In [4]:
import re


In [5]:
file1 = open('input.txt', 'r')
lines = file1.readlines()


In [6]:
seeds = []
seed_groups = []
maps = {}
current_map = ()
for line in lines:
    if re.match(r'seeds:', line) is not None:
        seeds_output = re.findall(r'\b\d+\b', line)
        seeds = seeds_output
        for i in range(0, len(seeds_output), 2):
            seed_groups.append((int(seeds_output[i]), int(seeds_output[i+1])))
    if line == "\n" or not line:
        continue

    if re.match(r'(\w*)-to-(\w*) map', line) is not None:
        current_map =re.match(r'(\w*)-to-(\w*) map', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"] = {}
        maps[f"{current_map[1]}-to-{current_map[0]}"] = {}
        continue

    if re.match(r'^\d+ \d+ \d+$', line) is not None:
        (elem1, elem2, cnt) = re.match(r'^(\d+) (\d+) (\d+)$', line).groups()
        maps[f"{current_map[0]}-to-{current_map[1]}"][int(elem2)] = {"diff": int(elem1) - int(elem2), "cnt": int(cnt)}
        maps[f"{current_map[1]}-to-{current_map[0]}"][int(elem1)] = {"diff": int(elem2) - int(elem1), "cnt": int(cnt)}


In [7]:
def get_value(dict, key):
    key_under = max([i for i in list(dict.keys()) if i <= key], default=None)
    if key_under is None:
        return key
    if dict[key_under]["cnt"] + key_under < key:
        return key
    return key+dict[key_under]["diff"]


In [8]:
min_value = 999999999999999999999999999999999999
seed_details = {}

for seed in seeds:
    soil = get_value(maps["seed-to-soil"], int(seed))
    fertilizer = get_value(maps["soil-to-fertilizer"], soil)
    water = get_value(maps["fertilizer-to-water"], fertilizer)
    light = get_value(maps["water-to-light"], water)
    temperature = get_value(maps["light-to-temperature"], light)
    humidity = get_value(maps["temperature-to-humidity"], temperature)
    location = get_value(maps["humidity-to-location"], humidity)
    if location < min_value:
        min_value = location
    seed_details[seed] = {
        "seed": seed,
        "soil": soil,
        "fertilizer": fertilizer,
        "water": water,
        "light": light,
        "temperature": temperature,
        "humidity": humidity,
        "location": location
    }
print(f"part1: {min_value}")


part1: 35


In [9]:
current_location = 0 # starting here to save time, but did multiple runs to find the right starting point
found = False
while not found:
    if current_location % 1000000 ==0:
        print(f"location:{current_location}")
    humidity = get_value(maps["location-to-humidity"], current_location)
    temperature = get_value(maps["humidity-to-temperature"], humidity)
    light = get_value(maps["temperature-to-light"], temperature)
    water = get_value(maps["light-to-water"], light)
    fertilizer = get_value(maps["water-to-fertilizer"], water)
    soil = get_value(maps["fertilizer-to-soil"], fertilizer)
    seed = get_value(maps["soil-to-seed"], soil)
    for seed_tupple in seed_groups:
        if seed_tupple[0] <= seed <= seed_tupple[0] + seed_tupple[1]:
            print(f"part2 location:{current_location} seed:{seed}")
            found = True
            break

    current_location += 1



location:0
part2 location:46 seed:82


In [9]:
#multi process approach
import multiprocess


pool_size = 20
increment = 340000000/20

# define worker function before a Pool is instantiated
def worker(entry):
    current_location = entry * increment
    found = False
    seed = 0
    initial_location = current_location

    while not found and current_location<=initial_location+increment:
        humidity = get_value(maps["location-to-humidity"], current_location)
        temperature = get_value(maps["humidity-to-temperature"], humidity)
        light = get_value(maps["temperature-to-light"], temperature)
        water = get_value(maps["light-to-water"], light)
        fertilizer = get_value(maps["water-to-fertilizer"], water)
        soil = get_value(maps["fertilizer-to-soil"], fertilizer)
        seed = get_value(maps["soil-to-seed"], soil)
        for seed_tupple in seed_groups:
            if seed_tupple[0] <= seed <= seed_tupple[0] + seed_tupple[1]:
                return current_location

        current_location += 1

pool = Pool(processes=pool_size)

variable = Value('i', 0)

pool = multiprocess.Pool(processes = pool_size)
output_from_processes = pool.map(worker, range(pool_size))


print(f"part 2: min location {min(value for value in output_from_processes if value is not None)}")


part 2: min location 219529182.0


In [101]:
def get_value_range(dict, ranges, output_ranges = []):
    print("------------------")
    print(f"dict {dict}")
    print(f"ranges {ranges}")
    print(f"output_ranges {output_ranges}")
    if len(ranges) == 0:
        print("LEAVING")
        return output_ranges
    key_under = max([i for i in list(dict.keys()) if i <= ranges[0][0]], default=None)
    if key_under is None:
        print("WILD")
        return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0], ranges[0][1])])
    range_length = dict[key_under]["cnt"]
    print(f"key_under {key_under}")
    print(f"range_length {range_length}")
    if key_under <= ranges[0][0] <= range_length + key_under: # if we start
        if ranges[0][0]+ranges[0][1] <= range_length + key_under: #fullRange is in the first range\
            print(f"IN")
            print(ranges)
            return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0] + dict[key_under]["diff"], ranges[0][1])])
        else:
            print(f"NOT")
            rr = ranges[1:] + [(ranges[0][0], ranges[0][0]-key_under)]
            print(rr)
            print(output_ranges + [(ranges[0][0]+ dict[key_under]["diff"], range_length + key_under)])
            return get_value_range(dict, rr,
                                output_ranges + [(ranges[0][0]+ dict[key_under]["diff"], range_length + key_under)]
                                    )
    else:
        print(f"ELSE")
        return get_value_range(dict, ranges[1:], output_ranges + [(ranges[0][0], ranges[0][1])])


In [102]:
min_value = 999999999999999999999999999999999999
seed_details = {}
final_location = []
# for seed_tupple in seed_groups:
print("========================")
print("seed")
print([seed_tupple])
print("========================")
soil = get_value_range(maps["seed-to-soil"], [seed_tupple])
print("========================")
print("soil")
print(soil)
print("========================")
fertilizer = get_value_range(maps["soil-to-fertilizer"], soil)
print("========================")
print("fer")
print(fertilizer)
print("========================")
water = get_value_range(maps["fertilizer-to-water"], fertilizer)
print("========================")
print("water")
print(water)
print("========================")
light = get_value_range(maps["water-to-light"], water)
print("========================")
print("light")
print(light)
print("========================")
temperature = get_value_range(maps["light-to-temperature"], light)
print("========================")
print("temperature")
print(temperature)
print("========================")
humidity = get_value_range(maps["temperature-to-humidity"], temperature)
print("========================")
print("humidity")
print(humidity)
print("========================")
location = get_value_range(maps["humidity-to-location"], humidity)
print("========================")
print("location")
print(location)
print("========================")
final_location.append(location)


seed
[(55, 13)]
------------------
dict {98: {'diff': -48, 'cnt': 2}, 50: {'diff': 2, 'cnt': 48}}
ranges [(55, 13)]
output_ranges []
key_under 50
range_length 48
IN
[(55, 13)]
------------------
dict {98: {'diff': -48, 'cnt': 2}, 50: {'diff': 2, 'cnt': 48}}
ranges []
output_ranges [(57, 13)]
LEAVING
soil
[(57, 13)]
------------------
dict {15: {'diff': -15, 'cnt': 37}, 52: {'diff': -15, 'cnt': 2}, 0: {'diff': 39, 'cnt': 15}}
ranges [(57, 13)]
output_ranges []
key_under 52
range_length 2
ELSE
------------------
dict {15: {'diff': -15, 'cnt': 37}, 52: {'diff': -15, 'cnt': 2}, 0: {'diff': 39, 'cnt': 15}}
ranges []
output_ranges [(57, 13)]
LEAVING
fer
[(57, 13)]
------------------
dict {53: {'diff': -4, 'cnt': 8}, 11: {'diff': -11, 'cnt': 42}, 0: {'diff': 42, 'cnt': 7}, 7: {'diff': 50, 'cnt': 4}}
ranges [(57, 13)]
output_ranges []
key_under 53
range_length 8
NOT
[(62, 5)]
[(53, 61)]
------------------
dict {53: {'diff': -4, 'cnt': 8}, 11: {'diff': -11, 'cnt': 42}, 0: {'diff': 42, 'cnt': 7}